In [1]:
from utils import (
    extract_nmf_feature,
    tweet_level_feature_generation,
    split_all_data,
    nn_tweet_only,
    add_nn_signal,
    add_feat,
    calculate_similarity,
    tweet_level_features,
    generate_sentiment_feature
)

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
random_seed = 3850

In [3]:
data_dict = split_all_data(.8, dev_test_split=.5, random_state=random_seed)
train_df = data_dict['train']
dev_df = data_dict['dev']
test_df = data_dict['test']
print('shape of train_df: ', train_df.shape)
print('shape of dev_df  : ', dev_df.shape)
print('shape of test_df : ', test_df.shape)

Loading the data...
Splitting the data...
shape of train_df:  (9460, 6)
shape of dev_df  :  (1183, 6)
shape of test_df :  (1183, 6)


In [4]:
train_df.sample(5, random_state=random_seed)

,ID,profile,tweet,neighbor,domain,label
5556,32447721,"{'id': '32447721 ', 'id_str': '32447721 ', 'na...",[I look forward to not being able to find a Nv...,"{'following': ['818668163466457088', '11995844...",[Entertainment],0
814,3017957005,"{'id': '3017957005 ', 'id_str': '3017957005 ',...",[RT @KendrickPerkins: Ladies and Gentlemen we ...,"{'following': ['63515922', '71415087', '293008...",[Politics],1
10475,264962685,"{'id': '264962685 ', 'id_str': '264962685 ', '...",[Nothing new comes from a standard response. #...,"{'following': ['760210372888563712', '86116147...",[Sports],1
7336,1074071841885417472,"{'id': '1074071841885417472 ', 'id_str': '1074...",[RT @DocSoph: Looking for unique &amp; innovat...,"{'following': ['1094029591054872576', '2288915...",[Sports],1
4999,1239433466460155904,"{'id': '1239433466460155904 ', 'id_str': '1239...",[RT @Sunflower_Films: Retweet this if you've l...,"{'following': ['1218302853502177280', '2425571...",[Entertainment],1


In [5]:
vectorizer, nn_model = nn_tweet_only(train_df, dev_df)

Prepping tweet data...
Vectorizing tweets...


In [ ]:
train_df = add_nn_signal(train_df, vectorizer, nn_model)
test_df = add_nn_signal(test_df, vectorizer, nn_model)

In [ ]:
train_df_stage_1 = add_feat(train_df)
test_df_stage_1 = add_feat(test_df)

In [ ]:
train_df_stage_1.sample(5, random_state=random_seed)

In [ ]:
# save point
train_df_stage_1.to_csv('Twibot-20/data_creation/train_df_stage_1.csv')
test_df_stage_1.to_csv('Twibot-20/data_creation/test_df_stage_1.csv')

In [ ]:
# train_df_stage_1 = pd.read_csv('Twibot-20/data_creation/train_df_stage_1.csv')
# test_df_stage_1 = pd.read_csv('Twibot-20/data_creation/test_df_stage_1.csv')

In [ ]:
train_df_stage_2 = calculate_similarity(train_df_stage_1['tweet'])
test_df_stage_2 = calculate_similarity(test_df_stage_1['tweet'])

In [ ]:
train_df_stage_2 = tweet_level_features(train_df_stage_2)
test_df_stage_2 = tweet_level_features(test_df_stage_2)

In [ ]:
train_df_stage_2 = generate_sentiment_feature(train_df_stage_2)
test_df_stage_2 = generate_sentiment_feature(test_df_stage_2)

In [ ]:
train_df_stage_2.sample(5, random_state=random_seed)

In [ ]:
# save point
train_df_stage_2.to_csv('Twibot-20/data_creation/train_df_stage_2.csv')
test_df_stage_2.to_csv('Twibot-20/data_creation/test_df_stage_2.csv')

In [ ]:
# train_df_stage_2 = pd.read_csv('Twibot-20/data_creation/train_df_stage_2.csv')
# test_df_stage_2 = pd.read_csv('Twibot-20/data_creation/test_df_stage_2.csv')

In [ ]:
train_df_stage_3 = tweet_level_feature_generation(train_df_stage_2)
test_df_stage_3 = tweet_level_feature_generation(test_df_stage_2)

In [ ]:
train_df_stage_3 = extract_nmf_feature(train_df_stage_3)
test_df_stage_3 = extract_nmf_feature(test_df_stage_3)

In [ ]:
train_df_stage_3.sample(5, random_state=random_seed)

In [ ]:
# save point
train_df_stage_3.to_csv('Twibot-20/data_creation/train_df_stage_3.csv')
test_df_stage_3.to_csv('Twibot-20/data_creation/test_df_stage_3.csv')